## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [ ]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

In [ ]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [ ]:
### Imprima os três primeiros registros da tabela tracks

In [ ]:
from sqlalchemy import select

Track = Base.classes.tracks
session = Session(engine)

# Consulta os três primeiros registros
tracks = session.scalars(select(Track).limit(3)).all()

for track in tracks:
    print(f"TrackId: {track.TrackId}, Name: {track.Name}, AlbumId: {track.AlbumId}, Milliseconds: {track.Milliseconds}")

In [ ]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.

In [ ]:
from sqlalchemy.orm import joinedload

Album = Base.classes.albums

# Consulta com join entre tracks e albums
tracks_with_albums = session.scalars(
    select(Track)
    .options(joinedload(Track.album))
    .limit(20)
).unique().all()

for track in tracks_with_albums:
    print(f"Track: {track.Name}, Album: {track.album.Title}")

In [ ]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.


In [ ]:
InvoiceItem = Base.classes.invoice_items

# Consulta com join entre invoice_items e tracks
sales = session.scalars(
    select(InvoiceItem)
    .options(joinedload(InvoiceItem.track))
    .limit(10)
).unique().all()

for sale in sales:
    print(f"Track: {sale.track.Name}, Quantity: {sale.Quantity}")

In [ ]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.

In [ ]:
from sqlalchemy import func

# Consulta agregada para contar vendas por faixa
top_tracks = session.execute(
    select(Track.Name, func.sum(InvoiceItem.Quantity).label('total_sales'))
    .join(InvoiceItem, InvoiceItem.TrackId == Track.TrackId)
    .group_by(Track.TrackId)
    .order_by(func.sum(InvoiceItem.Quantity).desc())
    .limit(10)
).all()

for track in top_tracks:
    print(f"Track: {track.Name}, Total Sales: {track.total_sales}")

In [ ]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists

In [ ]:
Artist = Base.classes.artists

# Consulta complexa com múltiplos joins
top_artists = session.execute(
    select(Artist.Name, func.sum(InvoiceItem.Quantity).label('total_sales'))
    .join(Album, Album.ArtistId == Artist.ArtistId)
    .join(Track, Track.AlbumId == Album.AlbumId)
    .join(InvoiceItem, InvoiceItem.TrackId == Track.TrackId)
    .group_by(Artist.ArtistId)
    .order_by(func.sum(InvoiceItem.Quantity).desc())
    .limit(10)
).all()

for artist in top_artists:
    print(f"Artist: {artist.Name}, Total Sales: {artist.total_sales}")